# Advent of Code, 7th Dec 2024

## Bridge Repair - Part 1

The Historians take you to a familiar rope bridge over a river in the middle of a jungle. The Chief isn't on this side of the bridge, though; maybe he's on the other side?

When you go to cross the bridge, you notice a group of engineers trying to repair it. (Apparently, it breaks pretty frequently.) You won't be able to cross until it's fixed.

You ask how long it'll take; the engineers tell you that it only needs final calibrations, but some young elephants were playing nearby and stole **all the operators** from their calibration equations! They could finish the calibrations if only someone could determine which test values could possibly be produced by placing any combination of operators into their calibration equations (your puzzle input).

For example:

```
190: 10 19
3267: 81 40 27
83: 17 5
156: 15 6
7290: 6 8 6 15
161011: 16 10 13
192: 17 8 14
21037: 9 7 18 13
292: 11 6 16 20
```

Each line represents a single equation. The test value appears before the colon on each line; it is your job to determine whether the remaining numbers can be combined with operators to produce the test value.

Operators are **always evaluated left-to-right**, not according to precedence rules. Furthermore, numbers in the equations cannot be rearranged. Glancing into the jungle, you can see elephants holding two different types of operators: **add** (+) and **multiply** (*).

Only three of the above equations can be made true by inserting operators:

* 190: 10 19 has only one position that accepts an operator: between 10 and 19. Choosing + would give 29, but choosing * would give the test value (10 * 19 = 190).
* 3267: 81 40 27 has two positions for operators. Of the four possible configurations of the operators, **two** cause the right side to match the test value: 81 + 40 * 27 and 81 * 40 + 27 both equal 3267 (when evaluated left-to-right)!
* 292: 11 6 16 20 can be solved in exactly one way: 11 + 6 * 16 + 20.

The engineers just need the **total calibration result**, which is the sum of the test values from just the equations that could possibly be true. In the above example, the sum of the test values for the three equations listed above is **3749**.

Determine which equations could possibly be true. **What is their total calibration result?**

### Read in the data

In [2]:
#!F#
let inputFilePath = "aoc-2024-12-07-puzzle-input.txt"

Read the lines from the file.

In [3]:
open System.IO

let lines: string list = File.ReadAllLines(inputFilePath) |> Array.toList
lines.Length

850

### Solve the problem

Each line needs to be split up into a result and a list of numbers that might be able to produce that result,
given the correct operators.

In [4]:
let parametersSeq = seq {
    for line in lines do
        let lineList = line.Split(':')
        let resultValue = lineList[0] |> Int64.Parse
        let numberList = lineList[1].Trim().Split(' ') |> Array.toList |> List.map (fun str -> Int32.Parse str)
        yield (resultValue, numberList)
}
let parametersMap = parametersSeq |> Map.ofSeq
parametersMap.Count

850

Now we check if we can produce the result values using the given numbers, in each case.

In [5]:
let rec calculatePossibleResults (numbers: int list) (currentValue: int64 option): int64 list =
    match numbers with
    | [] ->
        match currentValue with
        | None -> []
        | Some(value) -> [value]
    | head::tail ->
        match currentValue with
        | None -> calculatePossibleResults tail (Some head)
        | Some(value) -> (calculatePossibleResults tail (Some (value + (int64 head)))) @ (calculatePossibleResults tail (Some (value * (int64 head))))

let calculableResultSeq = seq {
    for resultValue in parametersMap.Keys do
        let numberList = parametersMap[resultValue]
        let possibleResults = calculatePossibleResults numberList None |> Set.ofList
        if (possibleResults |> Set.contains resultValue) then yield resultValue
}
let calculableResultList = calculableResultSeq |> Seq.toList

(calculableResultList.Length, calculableResultList |> List.sum)

Item1,238
Item2,975671981569


## Bridge Repair - Part 2

The engineers seem concerned; the total calibration result you gave them is nowhere close to being within safety tolerances. Just then, you spot your mistake: some well-hidden elephants are holding a **third type of operator**.

The **concatenation** operator (||) combines the digits from its left and right inputs into a single number. For example, 12 || 345 would become 12345. All operators are still evaluated left-to-right.

Now, apart from the three equations that could be made true using only addition and multiplication, the above example has three more equations that can be made true by inserting operators:

* 156: 15 6 can be made true through a single concatenation: 15 || 6 = 156.
* 7290: 6 8 6 15 can be made true using 6 * 8 || 6 * 15.
* 192: 17 8 14 can be made true using 17 || 8 + 14.

Adding up all six test values (the three that could be made before using only + and * plus the new three that can now be made by also using ||) produces the new **total calibration result** of **11387**.

Using your new knowledge of elephant hiding spots, determine which equations could possibly be true. **What is their total calibration result?**

### Solve the problem

In [6]:
let appendInt (val1: int64) (val2: int32): int64 =
    let intStr = val1.ToString() + val2.ToString()
    intStr |> Int64.Parse

In [7]:
appendInt (int64 123) 45

12345

In [8]:
let rec calculatePossibleResults2 (numbers: int list) (currentValue: int64 option): int64 list =
    match numbers with
    | [] ->
        match currentValue with
        | None -> []
        | Some(value) -> [value]
    | head::tail ->
        match currentValue with
        | None -> calculatePossibleResults2 tail (Some head)
        | Some(value) ->
            (calculatePossibleResults2 tail (Some (value + (int64 head)))) @
            (calculatePossibleResults2 tail (Some (value * (int64 head)))) @
            (calculatePossibleResults2 tail (Some (appendInt value head))) // '||' operator

let calculableResultSeq2 = seq {
    for resultValue in parametersMap.Keys do
        let numberList = parametersMap[resultValue]
        let possibleResults = calculatePossibleResults2 numberList None |> Set.ofList
        if (possibleResults |> Set.contains resultValue) then yield resultValue
}
let calculableResultList2 = calculableResultSeq2 |> Seq.toList

(calculableResultList2.Length, calculableResultList2 |> List.sum)

Item1,573
Item2,223472064194845
